# Web scraping data about countries

This is my first data analysis project, including scraping two websites using BeatufilSoup and Numpy with Pandas for data cleaning. The collected data is merged into a single dataframe and used for visualization in Power BI, available on GitHub.


The code below works as of 08/08/2023.<br>

Scraped websites:<br>
https://en.wikipedia.org/wiki/List_of_countries_by_stock_market_capitalization <br>
https://www.worldometers.info/world-population/population-by-country/

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
import numpy as np

In [ ]:
pd.options.display.max_rows = 999

## First table

In [ ]:
url1 = 'https://en.wikipedia.org/wiki/List_of_countries_by_stock_market_capitalization'

page1 = requests.get(url1)
soup1 = bs(page1.text, 'html')

In [ ]:
table1 = soup1.find_all('table')[0]

In [ ]:
print(table1)

In [ ]:
titles = table1.find_all('th')

In [ ]:
titles

In [ ]:
titles_names = [title.text.strip() for title in titles]

In [ ]:
titles_names

In [ ]:
#removing [digit] and adding spaces
clean_titles = []
for phrase in titles_names:
    clean_title = re.sub(r'\[\d*\]', '', phrase)
    clean_title = re.sub(r'\(', ' (', clean_title)
    clean_title = re.sub(r'cc', 'c c', clean_title)
    clean_titles.append(clean_title)
print(clean_titles)

In [ ]:
df1 = pd.DataFrame(columns = clean_titles)

In [ ]:
#names of columns
df1

In [ ]:
column_values1 =  table1.find_all('tr')

In [ ]:
#adding rows to a dataframe
for row in column_values1[1:]:
    row_data = row.find_all('td')
    row_values = [data.text.strip() for data in row_data]
    length = len(df1)
    df1.loc[length] = row_values

In [ ]:
df1

In [ ]:
#cleaning data
cols_to_check = df1.columns[1:]

df1[cols_to_check] = df1[cols_to_check].replace({',' : ''}, regex=True)
df1[cols_to_check] = df1[cols_to_check].replace({'\.\.\.' : '0'}, regex=True)
df1['Total market cap (in mil. US$)'] = df1['Total market cap (in mil. US$)'].replace({'\[\d*\]' : '', }, regex=True)

#first dataframe ready
df1.head()

## Second table

In [ ]:
url2 = 'https://www.worldometers.info/world-population/population-by-country/'

page2 = requests.get(url2)
soup2 = bs(page2.text, 'html')

In [ ]:
table2 = soup2.find('table')
table2

In [ ]:
table2_columns = table2.find_all('th')[1:]
table2_columns

In [ ]:
table2_columns_names = [col.text.strip() for col in table2_columns]
table2_columns_names

In [ ]:
df2 = pd.DataFrame(columns = table2_columns_names)
df2

In [ ]:
column2_values =  table2.find_all('tr')[1:]
column2_values

In [ ]:
#adding rows to a dataframe
for row in column2_values:
    row_data = row.find_all('td')[1:]
    row_values = [data.text.strip() for data in row_data]
    length = len(df2)
    df2.loc[length] = row_values
df2

In [ ]:
#removing % from values in dataframe
percent_columns2 = ['Yearly Change', 'Urban Pop %', 'World Share']
for col in percent_columns2:
    df2[col] = df2[col].str.strip('%')

df2

In [ ]:
#cleaning data
cols2 = df2.columns

df2[cols2] = df2[cols2].replace({',' : ''}, regex=True)
df2[cols2] = df2[cols2].replace({'N.A.' : '0'}, regex=True)
df2[cols2] = df2[cols2].replace({'' : '0'}, regex=True)

In [ ]:
df2

In [ ]:
#merging dataframes by countries
df3 = df1.merge(df2, left_on ='Country', right_on = 'Country (or dependency)')

In [ ]:
df3

In [ ]:
#countries that are in df1, but not in df2 
df1 [~df1['Country'].isin(df2['Country (or dependency)'])]

In [ ]:
#checking if above countries are in df2 
df_missing_countries = df1 [~df1['Country'].isin(df2['Country (or dependency)'])]
missing_countries = df_missing_countries['Country'].tolist()
df2_countries = df2['Country (or dependency)'].tolist()

for index, b in enumerate(df2_countries):
    for a in missing_countries:
        if a in b:
            print(list((index, b)))

In [ ]:
#replacing name of the country
df2.replace('Czech Republic (Czechia)', 'Czech Republic', inplace = True)
df2

In [ ]:
#another way of replacing name of the country
df2['Country (or dependency)'].iloc[120] = 'Palestine'
df2

In [ ]:
#dataframe with Czech Republic and Palestine
df3 = df1.merge(df2, left_on ='Country', right_on = 'Country (or dependency)')

In [ ]:
df3

In [ ]:
df3.loc[df3['Country'] == 'Palestine']

In [ ]:
df4 = df3[['Country','Total market cap (in mil. US$)','Number of domestic companies listed', 'Population (2023)', 'Land Area (Km²)']]


In [ ]:
#converting into numerical values
ignore = ['Country']
df4 = (df4.set_index(ignore, append=True)
        .astype(int)
        .reset_index(ignore))
df4.dtypes

#### Adding new columns

In [ ]:
df4['Capitalization per capita in USD'] = df4['Total market cap (in mil. US$)'] / df4['Population (2023)']
df4['Capitalization per capita in USD'] = df4['Capitalization per capita in USD'] * 1e6
df4['Capitalization per capita in USD'] = df4['Capitalization per capita in USD'].astype('int64')

In [ ]:
df4['Capitalization per area (km²) in USD'] = (df4['Total market cap (in mil. US$)'].div(df4['Land Area (Km²)'])) * 1e6
df4['Capitalization per area (km²) in USD'] = df4['Capitalization per area (km²) in USD'].astype(int)

In [ ]:
df4['Capitalization per area (km²) in USD'] = df4['Capitalization per area (km²) in USD'].astype('i')

In [ ]:
df4['number of people per domestic company'] = (df4['Population (2023)'].div(df4['Number of domestic companies listed']).replace(np.inf, 0))
df4['number of people per domestic company'] = df4['number of people per domestic company'].astype('int64')

In [ ]:
df4.sort_values(by = ['Number of domestic companies listed'], ascending=False).reset_index()

In [ ]:
df4.loc[df4['Country'] == 'Poland' ]

In [ ]:
df4

In [ ]:
df4.to_csv(r'D:\Data sets\web scrapping\final_dataframe1.csv', index = False )